In [1]:
import pandas as pd
import numpy as np

INICIALIZANDO

In [2]:
import json
def load_json():
    with open("config.json", "r") as f:
        return json.load(f)
    
# Carregar o JSON com as configurações necessárias
json = load_json()
team = json['team']

# INICIALIZADORES DE LISTAS
#------------------------------------------------------------------------------------------------------------------
# Columns
player_info = ['No.', 'Player', 'Age', 'Pos', 'G', 'GS', 'Wt (lb)', 'Wt (Kg - Approx)', 'Ht (Ft-In)', 'Ht (Mt - Approx)', 'AV', 'College/Univ', 'Player-additional']
player_addit = ['BirthDate', 'Yrs', 'Drafted (tm/rnd/yr)', 'Team']
players = player_info + player_addit
player_season = ['Team', 'Player-additional']

# Columns Stats
passers = ['QBrec', 'PassCmp', 'PassAtt', 'PassYds', 'PassTD', 'PassInt', 'PassAY/A', 'PassRate']
rushers = ['RushAtt','RushYds','RushTD']
receivers = ['Tgt', 'Rec', 'RecYds', 'RecTD']
defenders = ['DefInt', 'DefIntYds', 'DefIntTD', 'FF', 'Fmb', 'FR', 'FmbYds', 'FRTD', 'Sk', 'Comb', 'Solo', 'Ast', 'TFL', 'QBHits', 'Sfty']
kickers = ['0-19FGA', '0-19FGM', '20-29FGA', '20-29FGM', '30-39FGA', '30-39FGM', '40-49FGA', '40-49FGM', '50+FGA', '50+FGM', 'ScoringFGA', 'ScoringFGM', 'ScoringXPA', 'ScoringXPM' ]
punters = ['Pnt', 'PntYds', 'RetYds', 'PntLng', 'In20', 'PntBlck']
returners = ['PrRet', 'PrYds', 'PrTD','KrRet', 'KrYds', 'KrTD']
avs = ['approx_value_ol', 'approx_value_rush', 'approx_value_pass', 'approx_value_rec', 'approx_value_def', 'approx_value_ret', 'approx_value_kick', 'approx_value_punt']
columns_del = ['pos_multiplier', 'all_pro_multiplier', 'Ru_Bonus', 'Ru_Penalties', 'Pa_Bonus', 'Pa_Penalties', 'tkl_constant', 'all_pro_level', 'def_individual_points']
# Positions
tackles = ['T', 'OT', 'LT', 'RT']
guards_centers = ['G', 'OG', 'RG', 'LG', 'C', 'LS']
offensive_line = tackles + guards_centers
defensive_lineman = ['DL', 'RDE', 'LDE', 'DE', 'DT', 'RDT', 'LDT', 'NT']
linebackers = ['LB', 'OLB', 'ILB', 'MLB', 'LILB', 'RILB', 'LOLB', 'ROLB', 'LLB', 'RLB']
front_seven = defensive_lineman + linebackers
secondary = ['DB', 'CB', 'RCB', 'LCB', 'S', 'SS', 'FS']
defense = front_seven + secondary

CARREGANDO OS DATAFRAMES

In [3]:
#carregamento dos dataframes
#------------------------------------------------------------------------------------------------------------------

# Dataframe geral dos 32 times
df = pd.read_csv("data/raw/Stats/2023/PFR_AV_Calc.csv", sep=';', encoding='utf-8')

# Dataframe por Jogador de um time
df_team = pd.read_csv(f'data/raw/Teams/2023/{team}.csv', sep=',')

# Awards
df_PB = pd.read_csv('data/raw/Awards/2023/PB.csv', sep=',')
df_AP = pd.read_csv('data/raw/Awards/2023/AP.csv', sep=',')

# Dados brutos das Estatisticas
df_passing = pd.read_csv('data/raw/Stats/2023/Passing.csv', sep=',')
df_rushing = pd.read_csv('data/raw/Stats/2023/Rushing.csv', sep=',')
df_receiving = pd.read_csv('data/raw/Stats/2023/Receiving.csv', sep=',')
df_defense = pd.read_csv('data/raw/Stats/2023/Defense.csv', sep=',')
df_kicking = pd.read_csv('data/raw/Stats/2023/Kicking.csv', sep=',')
df_punting = pd.read_csv('data/raw/Stats/2023/Punting.csv', sep=',')
df_returning = pd.read_csv('data/raw/Stats/2023/Returning.csv', sep=',')


TRATANDO OS DATAFRAMES

In [4]:
# tratamento e manipulação dos dataframes
#------------------------------------------------------------------------------------------------------------------
# incluindo a coluna 'Team' com a variável 'team'
df_team['Team'] = team

# Convertendo o peso e a altura dos jogadores
df_team['Wt (Kg - Approx)'] = np.floor(df_team['Wt'] / 2.205)
df_team['Ht (Ft)'] = df_team['Ht'].str.split('-', expand=True)[0]
df_team['Ht (In)'] = df_team['Ht'].str.split('-', expand=True)[1]
df_team['Ht (Ft)'] = df_team['Ht (Ft)'].astype(int)
df_team['Ht (In)'] = df_team['Ht (In)'].astype(int)
df_team['Ht (Mt - Approx)'] = np.round(((df_team['Ht (Ft)']) + (df_team['Ht (In)'] / 12)) / 3.281, decimals=2)
df_team = df_team.drop(columns=['Ht (Ft)', 'Ht (In)'])

# renomeando colunas
df_team.rename(columns={'Wt': 'Wt (lb)', 'Ht':'Ht (Ft-In)'}, inplace=True)
df_kicking.rename(columns={'Tm': 'Team'}, inplace=True)
df_punting.rename(columns={'Tm': 'Team'}, inplace=True)
df_returning.rename(columns={'Tm': 'Team'}, inplace=True)
df_PB.rename(columns={'Tm': 'Team'}, inplace=True)
df_AP.rename(columns={'Tm': 'Team'}, inplace=True)
df_passing.rename(columns={'Sk': 'PassSk'}, inplace=True)
df_rushing.rename(columns={'1D': 'Rush1D'}, inplace=True)
df_rushing.rename(columns={'Fmb': 'RushFmb'}, inplace=True)
df_receiving.rename(columns={'1D': 'Rec1D'}, inplace=True)
df_receiving.rename(columns={'Fmb': 'RecFmb'}, inplace=True)
df_kicking.rename(columns={'TB': 'KOTB'}, inplace=True)
df_punting.rename(columns={'TB': 'PntTB'}, inplace=True)

positions = player_season
positions.append('Pos')

# df_players
df_players = pd.concat([df_passing[positions], df_rushing[positions]], ignore_index=True)
df_players = pd.concat([df_players, df_receiving[positions]], ignore_index=True)
df_players = pd.concat([df_players, df_defense[positions]], ignore_index=True)
df_players = pd.concat([df_players, df_kicking[positions]], ignore_index=True)
df_players = pd.concat([df_players, df_punting[positions]], ignore_index=True)
df_players = pd.concat([df_players, df_returning[positions]], ignore_index=True)
df_players = df_players.drop_duplicates()

# df_stats
df_stats = pd.merge(df_players, df_passing[player_season + passers], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_rushing[player_season + rushers], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_receiving[player_season + receivers], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_defense[player_season + defenders], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_kicking[player_season + kickers], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_punting[player_season + punters], on=player_season, how='left')
df_stats = pd.merge(df_stats, df_returning[player_season + returners], on=player_season, how='left')

# df_merged
df_merged = pd.merge(df_team[players], df_PB[['Team','Player-additional']], on='Player-additional', how='left')
df_merged = pd.merge(df_merged, df_AP[['Player-additional','All-pro teams']], on='Player-additional', how='left')

df_merged.rename(columns={"Team_x":"Team"}, inplace=True)
df_merged.rename(columns={"Team_y":"Pro-Bowl"}, inplace=True)
df_merged['Pro-Bowl'] = df_merged['Pro-Bowl'].replace(team, '1')
df_merged['Pro-Bowl'] = df_merged['Pro-Bowl'].fillna('0')
df_merged['All-pro teams'] = df_merged['All-pro teams'].fillna('0')
df_merged['All-pro teams'] = df_merged['All-pro teams'].str.replace(r'^\w+\: (\d).*?$', r'\1', regex=True)

df_merged = pd.merge(df_merged, df_stats, on=player_season, how='left')
df_merged = df_merged.dropna(subset=['Player'])
df_merged['Drafted (tm/rnd/yr)'] = df_merged['Drafted (tm/rnd/yr)'].fillna('Undrafted')
df_merged = df_merged.fillna(0)

AQUI COMEÇA OS CÁLCULOS

OFFENSE

In [5]:
# Variáveis
games = df_merged['G']
games_started = df_merged['GS']
rushTD = df['Rushing TD']
passTD = df['Passing TD']
fg = df['Scoring FGM']
turnovers = df['TO']
punts = df['Punting Pnt']
fga = df['Scoring FGA']

# Calculo offensive_points_per_drive
offensive_points_per_drive = (7*(rushTD+passTD) + 3*fg) / (rushTD + passTD + turnovers + punts + fga)

# avg offensive_points_per_drive
league_average_offensive_points_per_drive = offensive_points_per_drive.mean()

# team_offense_points
df['team_offense_points'] = 100 * (offensive_points_per_drive / league_average_offensive_points_per_drive)

# loc de um respectivo time pra retornar o 'team_offense_points' deste
team_offense_points = df.loc[df['Team'] == team, 'team_offense_points'].values


OFFENSE LINE

In [6]:
# função pos_mult
def pos_mult(row):
    row['pos_multiplier'] = 0.0

    # Offensive Tackles (Left ou Right)
    if row['Pos'] in tackles:
        row['pos_multiplier'] = 1.2

    # Offensive Guards (Left ou Right) e Centers
    if row['Pos'] in guards_centers:
        row['pos_multiplier'] = 1.0
    
    # Fullbacks
    if row['Pos'] == 'FB': 
        row['pos_multiplier'] = 0.3
    
    # Tight Ends
    if row['Pos'] == 'TE': 
        row['pos_multiplier'] = 0.2 

    return row

# Função all_pro_mult 
def all_pro_mult(row):
    row['all_pro_multiplier'] = 0.0

    # First Team All-Pro
    if (row['Pos'] in tackles) or (row['Pos'] in guards_centers):
        if row['All-pro teams'] == '1':
            row['all_pro_multiplier'] = 1.9

        # Second Team All-Pro
        elif row['All-pro teams'] == '2':
            row['all_pro_multiplier'] = 1.6

        # Pro Bowl
        elif (row['All-pro teams'] == '0') and (row['Pro-Bowl'] == '1'):
            row['all_pro_multiplier'] = 1.3

        # Default Value
        else:
            row['all_pro_multiplier'] = 1.0

    return row

# Calculando o team_points_for_o_line
team_points_for_o_line = 5 / 11 * team_offense_points

# pos_multiplier
df_merged = df_merged.apply(pos_mult, axis=1)

# all_pro_multiplier
df_merged = df_merged.apply(all_pro_mult, axis=1)

# Individual Points
ol_individual_points = (games + 5 * games_started * df_merged['pos_multiplier']) * df_merged['all_pro_multiplier']

# Soma dos pontos individuais
sum_individual_points = ol_individual_points.sum()

# Calcula o approx_value
df_merged['approx_value_ol'] = (ol_individual_points / sum_individual_points) * team_points_for_o_line

# Fill NaN values
df_merged['approx_value_ol'] = df_merged['approx_value_ol'].fillna(0.0)



SKILL OFFENSE

In [7]:
# Variáveis
team_offense_points = df.loc[df['Team'] == team, 'team_offense_points'].values
team_points_for_skill_positions = team_offense_points - team_points_for_o_line
team_rsh_yards = df.loc[df['Team'] == team, 'Rushing Yds'].values
team_total_yards = df.loc[df['Team'] == team, 'Yds'].values


RUSHERS

In [8]:
# team_points_for_rushers = team_points_for_skill_positions * (.22) * [(team_rsh_yards / team_total_yards ) / .37 ]
team_points_for_rushers = team_points_for_skill_positions * (0.22) * ((team_rsh_yards / team_total_yards ) / 0.37)
df_stats['RushY/A'] = df_stats['RushYds'] / df_stats['RushAtt']
league_rush_per_rb = df_stats.loc[df_stats['Pos'].isin(['RB']), 'RushY/A'].mean()
df_merged['RushY/A'] = df_merged['RushYds'] / df_merged['RushAtt']

# Pa_Bonus
df_merged['Ru_Bonus'] = np.where(
    (df_merged['RushAtt'] > 200) & (df_merged['RushY/A'] > league_rush_per_rb),
    0.5 * (df_merged['RushY/A'] - league_rush_per_rb),
    0.0
)

# Pa_Penalties
df_merged['Ru_Penalties'] = np.where(
    (df_merged['RushAtt'] > 200) & (df_merged['RushY/A'] < league_rush_per_rb),
    2 * (df_merged['RushY/A'] - league_rush_per_rb),
    0.0
)

# Approximated Value
df_merged['approx_value_rush'] = ((df_merged['RushYds']) / (team_rsh_yards) * team_points_for_rushers) + df_merged['Ru_Bonus'] + df_merged['Ru_Penalties']

# Fill NaN values
df_merged['approx_value_rush'] = df_merged['approx_value_rush'].fillna(0.0)

PASSERS

In [9]:
# Variáveis
passing_yards = df_merged['PassYds']
team_passing_yards = df.loc[df['Team'] == team, 'Passing Yds'].values
team_points_for_passers = (team_points_for_skill_positions - team_points_for_rushers) * 0.26
league_avg_pass_aya = df_stats.loc[df_stats['Pos'].isin(['QB']), 'PassAY/A'].mean()

# Pa_Bonus
df_merged['Pa_Bonus'] = np.where(
    (df_merged['PassAtt'] > 150) & (df_merged['PassAY/A'] > league_avg_pass_aya),
    0.5 * (df_merged['PassAY/A'] - league_avg_pass_aya),
    0.0
)

# Pa_Penalties
df_merged['Pa_Penalties'] = np.where(
    (df_merged['PassAtt'] > 150) & (df_merged['PassAY/A'] < league_avg_pass_aya),
    2 * (df_merged['PassAY/A'] - league_avg_pass_aya),
    0.0
)

# Approximated Value
df_merged['approx_value_pass'] = (passing_yards / team_passing_yards * team_points_for_passers) + df_merged['Pa_Bonus'] + df_merged['Pa_Penalties']

# Fill NaN values
df_merged['approx_value_pass'] = df_merged['approx_value_pass'].fillna(0.0)

RECEIVERS

In [10]:
# Variáveis
team_points_for_receivers = (team_points_for_skill_positions - team_points_for_rushers) * .74
receiving_yards = df_merged['RecYds']
team_receiving_yards = df_merged.loc[df_merged['Pos'].isin(['WR', 'TE']), 'RecYds'].sum()

# Approximated Value
df_merged['approx_value_rec'] = (receiving_yards) / (team_receiving_yards) * team_points_for_receivers

# Fill NaN values
df_merged['approx_value_rec'] = df_merged['approx_value_rec'].fillna(0.0)

DEFENSE

In [11]:
# Função tkl_constant_mult
def tkl_constant_mult(row):

    row['tkl_constant'] = 0
    # Defensive Line
    if row['Pos'] in defensive_lineman:
        row['tkl_constant'] = 0.6

    # linebackers
    if row['Pos'] in linebackers:
        row['tkl_constant'] = 0.3
        
    # Defensive Backs
    if row['Pos'] in secondary:
        row['tkl_constant'] = 0.0

    return row

# Função all_pro_level_mult
def all_pro_level_mult(row):

    row['all_pro_level'] = 0

    # First Team All-Pro
    if row['All-pro teams'] == '1':
        row['all_pro_level'] = 1.5

    # Second Team All-Pro
    if row['All-pro teams'] == '2':
        row['all_pro_level'] = 1.0

    # Pro Bowl
    if (row['All-pro teams'] == '0') and (row['Pro-Bowl'] == '1'):
        row['all_pro_level'] = 0.5
        
    return row

# Variáveis
# ------------------------------------------------------------------------------------------------------------------
oppRushTD = df['Opp Rushing TD']
oppPassTD = df['Opp Passing TD']
oppFg = df['Opp Scoring FGM']
oppTurnovers = df['Opp TO']
oppPunts = df['Opp Punting Pnt']
oppFga = df['Opp Scoring FGA']

defensive_points_allowed_per_drive = (7*(oppRushTD+oppPassTD) + 3*oppFg) / (oppRushTD + oppPassTD + oppTurnovers + oppPunts + oppFga)
league_defensive_points_allowed_per_drive = defensive_points_allowed_per_drive.mean()
m = defensive_points_allowed_per_drive / league_defensive_points_allowed_per_drive

# ------------------------------------------------------------------------------------------------------------------
# team_defense_points = 100 * [ (1 + 2*M - M2) / (2*M) ]
df['team_defense_points'] = 100 * ((1 + 2*m - m**2) / (2*m))
team_defense_points = df.loc[df['Team'] == team, 'team_defense_points'].values

games_played = df_merged['G']
games_started = df_merged['GS']
tackles = df_merged['Solo']
sacks = df_merged['Sk']
fumble_recoveries = df_merged['FR']
interceptions = df_merged['DefInt']
defensive_TDs = df_merged['DefIntTD'] + df_merged['FRTD']
# ------------------------------------------------------------------------------------------------------------------

team_points_for_front_7 = (2/3) * team_defense_points
team_points_for_secondary = (1/3) * team_defense_points

# tkl_constant
df_merged= df_merged.apply(tkl_constant_mult, axis=1)

# all_pro_level
df_merged= df_merged.apply(all_pro_level_mult, axis=1)

# Variáveis (Segunda Parte)
year_constant = 80
number_of_games_multiplier = 17 / 16
year_multiplier = year_constant * number_of_games_multiplier
all_pro_bonus = (df_merged['all_pro_level'])*(year_multiplier)

# individual_points = [(games played) + 5*(games started) + sacks + 
# 4*(fumble recoveries) + 4*(interceptions) + 5*(defensive TDs) + (tkl_constant)*(tackles)] + (all_pro_bonus),
df_merged['def_individual_points'] = (games_played + 5*games_started + sacks + 4*fumble_recoveries + 4*interceptions + 5*defensive_TDs + df_merged['tkl_constant'] * tackles) + all_pro_bonus

# Approximated Value
# Front Seven
sum_individual_points = df_merged.loc[df_merged['Pos'].isin(front_seven), 'def_individual_points'].sum()
df_merged.loc[df_merged['Pos'].isin(front_seven), 'approx_value_def'] = (df_merged['def_individual_points'] / sum_individual_points) * team_points_for_front_7

# Secondary
sum_individual_points = df_merged.loc[df_merged['Pos'].isin(secondary), 'def_individual_points'].sum()
df_merged.loc[df_merged['Pos'].isin(secondary), 'approx_value_def'] = (df_merged['def_individual_points'] / sum_individual_points) * team_points_for_secondary

# Fill NaN values
df_merged['approx_value_def'] = df_merged['approx_value_def'].fillna(0.0)

SPECIAL TEAMS

RETURNERS

In [12]:
#Every player gets one point of approx_value for each kick or punt return TD.
pr_kr_td = df_merged['KrTD'] + df_merged['PrTD']

# Approximated Value
df_merged['approx_value_ret'] = pr_kr_td

# Fill NaN values
df_merged['approx_value_ret'] = df_merged['approx_value_ret'].fillna(0.0)

KICKERS

In [13]:
# Variaveis
xpm = df_merged['ScoringXPM']
xpa = df_merged['ScoringXPA']
fgm = df_merged['ScoringFGM']
fga = df_merged['ScoringFGA']

fgm0_19 =  df_merged['0-19FGM']
fgm20_29 =  df_merged['20-29FGM']
fgm30_39 =  df_merged['30-39FGM']
fgm40_49 =  df_merged['40-49FGM']
fgm50_plus =  df_merged['50+FGM']

fga0_19 =  df_merged['0-19FGA']
fga20_29 =  df_merged['20-29FGA']
fga30_39 =  df_merged['30-39FGA']
fga40_49 =  df_merged['40-49FGA']
fga50_plus =  df_merged['50+FGA']

# Pct Kicking League
pct_xp = df['Scoring XPM'] / df['Scoring XPA']
pct_fg0_19 = df['0-19 FGM'] / df['0-19 FGA']
pct_fg20_29 = df['20-29 FGM'] / df['20-29 FGA']
pct_fg30_39 = df['30-39 FGM'] / df['30-39 FGA']
pct_fg40_49 = df['40-49 FGM'] / df['40-49 FGA']
pct_fg50_plus = df['50+ FGM'] / df['50+ FGA']

# Avg Pct League
lg_xp_pct = pct_xp.mean() 
lg_fg1_pct = pct_fg0_19.mean() 
lg_fg2_pct = pct_fg20_29.mean()
lg_fg3_pct = pct_fg30_39.mean()
lg_fg4_pct = pct_fg40_49.mean()
lg_fg5_pct = pct_fg50_plus.mean()

# Variáveis PAA
PAA_xp = xpm - xpa * lg_xp_pct
PAA_fg1 = 3 * (fgm0_19 - fga0_19 * lg_fg1_pct)
PAA_fg2 = 3 * (fgm20_29 - fga20_29 * lg_fg2_pct)
PAA_fg3 = 3 * (fgm30_39 - fga30_39 * lg_fg3_pct)
PAA_fg4 = 3 * (fgm40_49 - fga40_49 * lg_fg4_pct)
PAA_fg5 = 3 * (fgm50_plus - fga50_plus * lg_fg5_pct)
PAA_fg_u = 0.0

# PAA_total
PAA_total = PAA_xp + PAA_fg1 + PAA_fg2 + PAA_fg3 + PAA_fg4 + PAA_fg5 + PAA_fg_u

#k_playing_time = xpa + 3 * fga
k_playing_time = xpa + 3 * fga
team_k_playing_time = k_playing_time.sum()

#pct_team_playing_time = k_playing_time / team_k_playing_time
kick_pct_team_playing_time = k_playing_time / team_k_playing_time

# Approximated Value
kick_avg_AV = (3.125 / 16) * 17 * kick_pct_team_playing_time
kick_raw_AV = kick_avg_AV + (PAA_total / 5)
df_merged['approx_value_kick'] = 16 * (kick_raw_AV / 17)

# Fill NaN values
df_merged['approx_value_kick'] = df_merged['approx_value_kick'].fillna(0.0)

PUNTERS

In [14]:
# League Adj Punt Y/A
df['lg_adj_punt_ypa'] = (df['Punting Yds'] - 13 * df['Punting Blck']) / (df['Punting Pnt'] + df['Punting Blck'])

# Variáveis
lg_adj_punt_ypa = df['lg_adj_punt_ypa'].mean()
punt = df_merged['Pnt']
punt_yds = df_merged['PntYds']
punt_blocked = df_merged['PntBlck']
team_punt_blocked = df_merged['PntBlck'].sum()
team_punt = df_merged['Pnt'].sum()

# adj_punt_ypa = (punt_yds - 13 * punt_blocked) / (punt + punt_blocked)
df_merged['adj_punt_ypa'] = (punt_yds - 13 * punt_blocked) / (punt + punt_blocked)
adj_punt_ypa = df_merged['adj_punt_ypa']

# adj_punt_yds_above_avg = (punt + punt_blocked) * (adj_punt_ypa - lg_adj_punt_ypa)
adj_punt_yds_above_avg = (punt + punt_blocked) * (adj_punt_ypa - lg_adj_punt_ypa)

#pct_team_playing_time = (punt + punt_blocked) / (team_punt + team_punt_blocked)]
punt_pct_team_playing_time = (punt + punt_blocked) / (team_punt + team_punt_blocked)

# Approximated Value
punt_avg_AV = (2.1875 / 16) * 17 * punt_pct_team_playing_time
punt_raw_AV = punt_avg_AV + (adj_punt_yds_above_avg / 200)
df_merged['approx_value_punt'] = 16 * (punt_raw_AV / 17)

# Fill NaN values
df_merged['approx_value_punt'] = df_merged['approx_value_punt'].fillna(0.0)

FINALIZANDO

In [15]:
# Preencher todos os valores Nan
df_merged = df_merged.fillna(0)
# Remover colunas desnecessárias
df_merged = df_merged.drop(columns=columns_del)
av_sum = 0
for av in avs:
    av_sum += df_merged[av]

df_merged['AV_Sum'] = av_sum
df_merged['AV_Sum'] = np.round(av_sum)

df_merged = df_merged.drop(columns=avs)
def is_equals(row):
    row['Is_Equals'] = True if row['AV'] == row['AV_Sum'] else False

    return row

df_merged['AV_Diff'] = df_merged['AV_Sum'] - df_merged['AV']
df_merged = df_merged.apply(is_equals, axis=1)
df_merged = df_merged.sort_values(by='No.')
sel_columns = ['No.', 'Player', 'G', 'GS', 'Pos', 'Pro-Bowl','All-pro teams','AV', 'AV_Sum', 'AV_Diff']
df_merged[sel_columns].loc[(df_merged['Is_Equals'] != True) & (df_merged['AV_Diff'] != 0)].head(20)

,No.,Player,G,GS,Pos,Pro-Bowl,All-pro teams,AV,AV_Sum,AV_Diff
7,4,James Cook,17,13,RB,1,0,13,12.0,-1.0
28,7,Taron Johnson,17,17,CB,0,2,12,10.0,-2.0
38,8,Sam Martin,17,0,P,0,0,1,2.0,1.0
54,10,Khalil Shakir,17,10,WR,0,0,6,7.0,1.0
23,11,Deonte Harty,16,1,WR,0,0,2,3.0,1.0
8,13,Gabriel Davis,17,17,WR,0,0,7,8.0,1.0
10,14,Stefon Diggs,17,17,WR,1,0,12,13.0,1.0
11,25,Tyrel Dodson,17,10,LB,0,0,6,7.0,1.0
60,27,Tre'Davious White,4,4,CB,0,0,2,1.0,-1.0
61,42,Dorian Williams,17,2,LB,0,0,2,3.0,1.0
